# Kaggle Santander competition

In [1]:
# general & data analysis imports
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
# from imblearn.over_sampling import SMOTE

import lightgbm as lgb
import time

from hyperopt import hp, tpe
from hyperopt.fmin import fmin




## data load

In [24]:
train_dataset=pd.read_csv('train.csv')
test_dataset=pd.read_csv('test.csv')

## data analysis

In [ ]:
train_dataset.info()

In [ ]:
train_dataset.describe()

In [ ]:
train_dataset.columns

In [ ]:
train_dataset.head()

In [ ]:
train_dataset.isnull().sum().sum()

## data preprocessing

In [25]:
# remove dependent variable from train set to have the same structure as test set
df_target=train_dataset['target'].copy()
df_train=train_dataset.drop(['ID_code','target'], axis=1)
df_test=test_dataset.drop('ID_code', axis=1)

In [ ]:
#sm = SMOTE(random_state=1)
#X_train, y_train = sm.fit_resample(X_train, y_train)
#X_val, y_val = sm.fit_resample(X_val, y_val)

## training, tuning and evaluation - random search
random hyperparameters search, without kfold

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(df_train.values,df_target.values,test_size=0.2,random_state=1, shuffle=True)
X_test=df_test.values

In [ ]:
#lgbm parameters values for random search
param_grid = dict(
         objective =  ['binary'],
         learning_rate = np.logspace(-3, -1, num=50, base=10.0),
         feature_fraction = np.logspace(-2, -1, num=50, base=10.0),
         num_leaves = np.arange(10,30,2),
         min_data_in_leaf = np.arange(30,150,50),
         bagging_fraction = np.arange(0.3,0.95,0.01),
         bagging_freq = np.arange(3, 30, 5),
         max_depth = [-1],
         boosting_type = ['gbdt'],
         metric = ['auc'],
         min_sum_hessian_in_leaf = np.logspace(-4, 2, num=50, base=10.0),
         n_jobs = [-1],
         num_round = [2500]
)

In [ ]:
def random_search(param_grid, X_train, X_val, y_train, y_val, iterations):
    train_set = lgb.Dataset(X_train, label=y_train)
    val_set = lgb.Dataset(X_val, label=y_val)
    param_list=list(param_grid.keys())
    metrics_list=['ROC_train','ROC_val','ROC_diff']
    logging_list=param_list+metrics_list
    results=[]
    try:
        for i in range(iterations):
            print(f'iteration {i+1} of {iterations}')
            # randomly select parameters
            param = dict()
            for key in param_grid:
                param[key] = np.random.choice(param_grid[key])
            print(f'selected params:{param}')
            # train the model
            clf = lgb.train(param, train_set, valid_sets=[train_set,val_set], verbose_eval=500,early_stopping_rounds = 400)
            # calculate & log statistics
            y_train_proba=clf.predict(X_train)
            y_val_proba=clf.predict(X_val)
            param['ROC_train']=roc_auc_score(y_train,y_train_proba)
            param['ROC_val']=roc_auc_score(y_val,y_val_proba)
            param['ROC_diff']=param['ROC_train']-param['ROC_val']
            logging_list
            # log results
            result_line=[]
            # log parameters
            for key in logging_list:
                result_line.append(param[key])
            results.append(result_line)
    except(KeyboardInterrupt):
        pass
    # save results to file
    result_df=pd.DataFrame(results, columns=logging_list)
    result_df.to_csv('hp_search.csv', index=False)

        

        
        

In [ ]:
random_search(param_grid, X_train, X_val, y_train, y_val, 400)

## training, tuning and evaluation - hyperopt
 hyperparameters search by hyperopt

In [4]:
X_train,X_val,y_train,y_val=train_test_split(df_train.values,df_target.values,test_size=0.15,random_state=1, shuffle=True)
X_test=df_test.values
train_set = lgb.Dataset(X_train, label=y_train)
val_set = lgb.Dataset(X_val, label=y_val)

In [36]:
space= { 'objective': 'binary', 
         'boosting_type': 'gbdt',
         'metric': 'auc',
         'max_depth': -1,
         'learning_rate': 0.04,
         # 'max_depth':hp.quniform('max_depth', 4, 14, 1),
         'feature_fraction': hp.uniform('feature_fraction',0.01, 0.2),
         'bagging_fraction': hp.uniform('bagging_fraction',0.2, 0.9),
         'num_leaves': hp.quniform('num_leaves', 10, 20, 1),
         'min_data_in_leaf': hp.quniform('min_data_in_leaf', 75, 85, 1),
         'bagging_freq': hp.quniform('bagging_freq', 3, 10, 1),
         'n_jobs': -1,
         'num_round': 3000
}

In [37]:
def check_model(param):
    # print received params
    # print(f'received params: {param}')
    # convert to integers
    integer_params=['max_depth', 'num_leaves', 'min_data_in_leaf', 'bagging_freq']
    for p in integer_params:
        param[p]=int(param[p])
    print(f'corrected params: {param}')
    # train clasiffier
    clf = lgb.train(param, train_set, valid_sets=[train_set,val_set], verbose_eval=1000,early_stopping_rounds = 400)
    # calculate ROC (more is better)
    y_val_proba=clf.predict(X_val)
    roc=roc_auc_score(y_val,y_val_proba)
    # return optimization result (less is better)
    return 1-roc


In [38]:
best_classifier = fmin(check_model,space,algo=tpe.suggest,max_evals=150)
print(best_classifier)


corrected params: {'bagging_fraction': 0.5929718721408997, 'bagging_freq': 3, 'boosting_type': 'gbdt', 'feature_fraction': 0.16058925971284546, 'learning_rate': 0.04, 'max_depth': -1, 'metric': 'auc', 'min_data_in_leaf': 85, 'n_jobs': -1, 'num_leaves': 19, 'num_round': 3000, 'objective': 'binary'}
Training until validation scores don't improve for 400 rounds.                                                         
[1000]	training's auc: 0.94907	valid_1's auc: 0.893897                                                                 
Early stopping, best iteration is:                                                                                     
[1331]	training's auc: 0.959086	valid_1's auc: 0.895009
corrected params: {'bagging_fraction': 0.5676401883525091, 'bagging_freq': 9, 'boosting_type': 'gbdt', 'feature_fraction': 0.04237930286436834, 'learning_rate': 0.04, 'max_depth': -1, 'metric': 'auc', 'min_data_in_leaf': 76, 'n_jobs': -1, 'num_leaves': 16, 'num_round': 3000, 'objectiv

C:\Users\tslow\Anaconda3\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



Training until validation scores don't improve for 400 rounds.                                                         
[1000]	training's auc: 0.934964	valid_1's auc: 0.894082                                                                
[2000]	training's auc: 0.955789	valid_1's auc: 0.89616                                                                 
Early stopping, best iteration is:                                                                                     
[1959]	training's auc: 0.955047	valid_1's auc: 0.896251
corrected params: {'bagging_fraction': 0.541424121691912, 'bagging_freq': 6, 'boosting_type': 'gbdt', 'feature_fraction': 0.010631911708499909, 'learning_rate': 0.04, 'max_depth': -1, 'metric': 'auc', 'min_data_in_leaf': 76, 'n_jobs': -1, 'num_leaves': 18, 'num_round': 3000, 'objective': 'binary'}
  1%|▋                                               | 2/150 [01:11<1:28:24, 35.84s/it, best loss: 0.05504867411840997]

C:\Users\tslow\Anaconda3\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



Training until validation scores don't improve for 400 rounds.                                                         
[1000]	training's auc: 0.938527	valid_1's auc: 0.888897                                                                
[2000]	training's auc: 0.952648	valid_1's auc: 0.893871                                                                
[3000]	training's auc: 0.963362	valid_1's auc: 0.894439                                                                
Did not meet early stopping. Best iteration is:                                                                        
[3000]	training's auc: 0.963362	valid_1's auc: 0.894439
corrected params: {'bagging_fraction': 0.8223535722910251, 'bagging_freq': 8, 'boosting_type': 'gbdt', 'feature_fraction': 0.05136731313686845, 'learning_rate': 0.04, 'max_depth': -1, 'metric': 'auc', 'min_data_in_leaf': 82, 'n_jobs': -1, 'num_leaves': 14, 'num_round': 3000, 'objective': 'binary'}
  2%|▉                                       

...

Training until validation scores don't improve for 400 rounds.                                                         
[1000]	training's auc: 0.948216	valid_1's auc: 0.893799                                                                
Early stopping, best iteration is:                                                                                     
[1483]	training's auc: 0.962158	valid_1's auc: 0.895594
corrected params: {'bagging_fraction': 0.7465902585104937, 'bagging_freq': 8, 'boosting_type': 'gbdt', 'feature_fraction': 0.12484661093913012, 'learning_rate': 0.04, 'max_depth': -1, 'metric': 'auc', 'min_data_in_leaf': 79, 'n_jobs': -1, 'num_leaves': 19, 'num_round': 3000, 'objective': 'binary'}
 99%|█████████████████████████████████████████████▍| 148/150 [2:10:53<01:36, 48.13s/it, best loss: 0.04062706349138234]

C:\Users\tslow\Anaconda3\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



Training until validation scores don't improve for 400 rounds.                                                         
[1000]	training's auc: 0.948223	valid_1's auc: 0.893377                                                                
[2000]	training's auc: 0.973447	valid_1's auc: 0.895021                                                                
Early stopping, best iteration is:                                                                                     
[1616]	training's auc: 0.965174	valid_1's auc: 0.895223
corrected params: {'bagging_fraction': 0.8992580503177581, 'bagging_freq': 6, 'boosting_type': 'gbdt', 'feature_fraction': 0.13684603175864024, 'learning_rate': 0.04, 'max_depth': -1, 'metric': 'auc', 'min_data_in_leaf': 82, 'n_jobs': -1, 'num_leaves': 19, 'num_round': 3000, 'objective': 'binary'}
 99%|█████████████████████████████████████████████▋| 149/150 [2:11:37<00:46, 46.95s/it, best loss: 0.04062706349138234]

C:\Users\tslow\Anaconda3\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



Training until validation scores don't improve for 400 rounds.                                                         
[1000]	training's auc: 0.950139	valid_1's auc: 0.892696                                                                
[2000]	training's auc: 0.974902	valid_1's auc: 0.894597                                                                
Early stopping, best iteration is:                                                                                     
[2002]	training's auc: 0.974949	valid_1's auc: 0.894612
100%|██████████████████████████████████████████████| 150/150 [2:12:35<00:00, 50.14s/it, best loss: 0.04062706349138234]
{'bagging_fraction': 0.7755883925843395, 'bagging_freq': 4.0, 'feature_fraction': 0.11619997622598302, 'min_data_in_leaf': 85.0, 'num_leaves': 19.0}


## final training
Kfold with best found hyperparameters. Predicted probabilities are mean of predictions from all folds.

In [39]:
X_test=df_test.values

In [41]:
# 0.900
# best_param = {'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.02, 'feature_fraction': 0.023,
#              'num_leaves': 10, 'min_data_in_leaf': 80, 'bagging_fraction': 0.46, 'bagging_freq': 18, 
#              'min_sum_hessian_in_leaf': 1.45, 'num_rounds': 5000, 'verbose': 1}
# fold_n=4
best_param = {'objective': 'binary', 'metric': 'auc', 'bagging_fraction': 0.7755883925843395, 'bagging_freq': 4,
              'feature_fraction': 0.11619997622598302, 'min_data_in_leaf': 85, 'num_leaves': 19,
              'learning_rate': 0.04, 'max_depth': -1, 'num_rounds': 5000, 'verbose': 1}
        # , 'device': 'gpu', 'gpu_use_dp': False}
fold_n=5

# predicted probabilities on test set (competition set)
y_probs = np.zeros(len(df_test.values))
folds = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=30)
for i, (train_index, valid_index) in enumerate(folds.split(df_train,df_target)):
    tic=time.time()
    print(f'Calculating fold {i+1}/{fold_n}...')
    train_set = lgb.Dataset(df_train.iloc[train_index], label=df_target.iloc[train_index])
    val_set = lgb.Dataset(df_train.iloc[valid_index], label=df_target.iloc[valid_index])
    clf = lgb.train(best_param, train_set, valid_sets=[train_set, val_set], verbose_eval=500, early_stopping_rounds = 400)
    # y_probs += clf.predict(df_test.values, num_iteration=clf.best_iteration)/fold_n
    y_probs += clf.predict(df_test.values)/fold_n
    toc=time.time()
    print(f'Fold {i+1} calcutated in {toc-tic}.')


Calculating fold 1/5...


C:\Users\tslow\Anaconda3\lib\site-packages\lightgbm\engine.py:113: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 400 rounds.
[500]	training's auc: 0.950332	valid_1's auc: 0.895778


KeyboardInterrupt: 

In [ ]:
#y_val_probs=clf.predict(X_val)
#y_val_preds=np.where(y_val_probs>0.5,1,0)

In [ ]:
#roc_auc_score(y_val, y_val_probs)

In [ ]:
#y_probs=clf.predict(X_test)

In [ ]:
y_probs


## submision

In [21]:
submission_df = pd.DataFrame({"ID_code":test_dataset["ID_code"].values})
submission_df["target"] = y_probs
submission_df.to_csv("submission.csv", index=False)